# Analysis of cell nuclei morphology

### I - Preprocessing

In this section an example how to generate single nuclei from a segmented and labbeled image is provided.
For this we use a **format_nuclei(..)**, which takes the following variables as input:
- objects : bounding box for each detected and labelled object in the image
- num_object: index of object to be cropped
- segmented_image: segmented image (binary)
- labelled_image: labelled segmented image
- image: original image
- file: filename
- cropsize: size of crop (+/-  average radius of the nuclei (in pixels) + 15% of that length )

In [23]:
#packages
import numpy as np
from skimage import io,img_as_ubyte
from scipy.ndimage import measurements,find_objects,interpolation,label

In [34]:
def format_nuclei(file,image,segmented_image,labelled_image,bb_object,index_object,cropsize):    
            nuc=segmented_image[bb_object[index_object]]
            mask=nuc==0
            crop_1=image[:,:,:,0]
            crop_1=crop_1[bb_object[index_object]]
            crop_1[mask]=0
            crop_1=np.pad(crop_1,((cropsize,cropsize),(cropsize,cropsize),(cropsize,cropsize)),'constant')
            crop_2=image[:,:,:,1]
            crop_2=crop_2[bb_object[index_object]]
            crop_2[mask]=0
            crop_2=np.pad(crop_2,((cropsize,cropsize),(cropsize,cropsize),(cropsize,cropsize)),'constant')
            nuc=np.pad(nuc,((cropsize,cropsize),(cropsize,cropsize),(cropsize,cropsize)),'constant')
            cm=measurements.center_of_mass(nuc)
            cmi=int(cm[0]),int(cm[1]),int(cm[2])
            binary=nuc[(cmi[0]-cropsize):(cmi[0]+cropsize),(cmi[1]-cropsize):(cmi[1]+cropsize),(cmi[2]-cropsize):(cmi[2]+cropsize)] 
            ch_1=crop_1[(cmi[0]-cropsize):(cmi[0]+cropsize),(cmi[1]-cropsize):(cmi[1]+cropsize),(cmi[2]-cropsize):(cmi[2]+cropsize)]  
            ch_2=crop_2[(cmi[0]-cropsize):(cmi[0]+cropsize),(cmi[1]-cropsize):(cmi[1]+cropsize),(cmi[2]-cropsize):(cmi[2]+cropsize)]  
            new_img=np.stack((img_as_ubyte(binary),img_as_ubyte(ch_1),img_as_ubyte(ch_2)))
            filename=file[:-4]+'_'+str(index_object+1)+'.tif'
            io.imsave((filename),new_img)
            return new_img    

The image we load is in the format `image[z,x,y,channels]`, make sure you have the same. If not, you can use [`np.reshape()`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html). 

In [37]:
#path to the file
file='./crop_sn/10_11_21_WT_C-D_A_1_gb_05.tif'

image = io.imread(file)
#pixel spacing in image, xy pixel = 0.15µm and zy = 0.26µm
spacing = 0.26/0.15
image=interpolation.zoom(image, (spacing,1,1,1),order=1)

#the segmented_image needs no spacing since it has already been processed in a previous step
segmented_image=io.imread('./crop_sn/segmented_image.tif')

#labelled image is created from segmented_image
labelled_image,labels=label(segmented_image>0)

#get the binding box coordinates from the objects in the labelled image
bb_objects=find_objects(labelled_image)

#cropsize, must be interger
cropsize=int(40+0.15*40)
    
nuclei=[format_nuclei(file,image,segmented_image,labelled_image,bb_objects,index_object,cropsize) for index_object in range(0,len(bb_objects))]   

In [38]:
# to continue the workflow, the images with the cropped nuclei need to have the same dimensions
# pls note that the cropped nuclei have the format image[channels,z,x,y].
dimensions=[print(nuc.shape) for nuc in nuclei]

(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)
(3, 92, 92, 92)


### II - Spherical transformation and Fourier transform

This part of the workflow transforms an n x n x n image to spherical coordinates before Fourier transform is applied. In brief, spherical transformation renders rotation invariance and Fouriertransformation followed by [`.flatten()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) transforms the image into a n*n*n long vector. For technical details refer to [Wagner et al., 2019](https://ieeexplore.ieee.org/document/8856734).

The following functions were coded with the help of Patrick Wagner the author of [Wagner et al., 2019](https://ieeexplore.ieee.org/document/8856734) (commented in code)

The function **GFT(..)** uses the following variables:
- file: filename of the input image
- channel: the channel of interest to pass to the function (max three channels). If 'NONE', than the array is assumed to be in image[z,x,y]
- size: An input image with dimension 40x40x40 yields a vector of length 64000 and is already sufficient as a morphological feature vector. One could try out lower resolution below 35 and compare the results. The smaller the image the, faster the function 


In [ ]:
#packages
import numpy as np
import os
import glob
from scipy import fftpack
from scipy.ndimage.interpolation import geometric_transform
from skimage import io

In [ ]:
#It is recommended to use size in a range of 40-50. 
#Eg. an input image with dimension 40x40x40 yields a vector of length 64000, this is already sufficient for morphological feature vector
#one could try out lower resolution below 35 and compare the results.

def open_file(file,channel,size):
    #if the input consists of one channel only
    if channel == 'NONE':
        img=io.imread(file)
        downscale=size/img.shape[1]
        img=resize(img, (img.shape[1]*downscale,img.shape[2]*downscale,img.shape[3]*downscale), anti_aliasing=False)
    #if the input has more than one channel
    else:
        img=io.imread(file)
        downscale=size/img.shape[1]
        img=resize(img, (3,img.shape[1]*downscale,img.shape[2]*downscale,img.shape[3]*downscale),anti_aliasing=False)
        img=img[channel,:,:,:] 
    return img
                
def img_shape(file,channel,size):
    if channel == 'NONE':
        shape_img=open_file(file,channel,size).shape
    else:
        shape_img=open_file(file,channel,size).shape
    return shape_img

#transformation to spherical coordinates
#code snippet kindly provided by Patrick Wagner
def to_spherical(img, shape, order=1):
                    D = shape[1] - 1
                    R = D/2.
                    def transform(coords):
                        [z,x,y] =  coords
                        phi = np.pi*z / R
                        theta = np.pi*y / R
                        rho =  x / 2.

                        x = rho*np.sin(theta)*np.cos(phi) + R
                        y = rho*np.sin(theta)*np.sin(phi) + R
                        z = rho*np.cos(theta) + R
                        return z,x,y

                    spherical = geometric_transform(img, transform, order=order, mode='nearest', prefilter=False)
                    return spherical

#generic Fourier transformation
#code snippet kindly provided by Patrick Wagner
def GFT(file,channel,size):
    print(file)
    result=abs(fftpack.fftshift(fftpack.fftn(tospherical(open_file(file,channel,size),img_shape(file,channel,size))))).flatten()
    return result
